In [5]:
from nba_api.stats.endpoints import commonplayerinfo
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats

In [17]:
# Create a dictionary of player ids with keys of Full name. This will allow for look-ups
# to get individual player data
player_ids = {item['full_name']:item['id'] for item in players.get_players()}

# Get Player of Week Data to generate list of players to get data for
pow_df = pd.read_csv('NBA_player_of_the_week.csv',parse_dates=['Date'])
pow_df.columns = [c.lower().replace(' ', '_') for c in pow_df.columns]
pow_set = set(pow_df['player'])

In [18]:
# Loop over the player of the week set - and get data from NBA API for each. 
players_not_found = []
comb_player_stats = pd.DataFrame()
for p, player in enumerate(pow_set):
    print("Working on", player)
    if player not in player_ids.keys():
        print(player, "not in id dictionary. Check naming convention.")
        players_not_found.append(player)
    else:
        player_stats = playercareerstats.PlayerCareerStats(player_id=player_ids[player])
        player_stats_df = player_stats.get_data_frames()[0]
        player_stats_df['PLAYER_NAME'] = player
        if p == 0:
            comb_player_stats = player_stats_df
        else:
            comb_player_stats = pd.concat([comb_player_stats, player_stats_df])


Working on Zach Randolph
Working on Ben Simmons
Working on Steve Francis
Working on Larry Johnson
Working on Lucious Harris
Working on Chris Webber
Working on Drazen Petrovic
Working on Cedric Ceballos
Working on Shareef Abdur-Rahim
Working on Alex English
Working on Shawn Kemp
Working on Victor Oladipo
Working on Al Harrington
Working on Dirk Nowitzki
Working on Vern Fleming
Working on Kenny Smith
Working on Hakeem Olajuwon
Working on Chris Paul
Working on Jeff Teague
Working on T.J. Ford
Working on Thaddeus Young
Working on Chris Wilcox
Working on Andrei Kirilenko
Working on George Hill
Working on Ricky Pierce
Working on Kevin Garnett
Working on Peja Stojakovic
Working on Mike Bibby
Working on Shawn Marion
Working on Kenny Anderson
Working on Nate Robinson
Working on Grant Hill
Working on Kobe Bryant
Working on Terry Teagle
Working on Fat Lever
Fat Lever not in id dictionary. Check naming convention.
Working on Kenneth Faried
Working on Baron Davis
Working on Tobias Harris
Working on

There were eight players not found when using the POW name to get a player id to use with the 
NBA API. These names are below:
1. Fat Lever  --> Found as Lafayette Lever
2. Bobby Hansen  --> Found as Bob Hansen
3. C.J. McCollum --> Found as CJ McCollum
4. Penny Hardaway --> Found as Anfernee Hardaway
5. J.R. Smith --> Found as JR Smith
6. Cliff Robinson --> Found as Clifford Robinson
7. World B. Free  --> Found as World Free
8. Steve Smith --> Found as Steven Smith (also a Stevin Smith - need to double check)

In [25]:
# These players weren't found in the ID look-up provided by the NBA API
# I will add them by hand
print(players_not_found)
updated_players = ["Lafayette Lever", "Bob Hansen", "CJ McCollum", "Anfernee Hardaway",
                  "JR Smith", "Clifford Robinson", "World Free", "Steven Smith"]
print(updated_players)

['Fat Lever', 'Bobby Hansen', 'C.J. McCollum', 'Penny Hardaway', 'J.R. Smith', 'Cliff Robinson', 'World B. Free', 'Steve Smith']
['Lafayette Lever', 'Bob Hansen', 'CJ McCollum', 'Anfernee Hardaway', 'JR Smith', 'Clifford Robinson', 'World Free', 'Steven Smith']


In [26]:
# Loop over the leftover players and get data from NBA API for each. 
for player in updated_players:
    print("Working on", player)
    if player not in player_ids.keys():
        print(player, "not in id dictionary. Check naming convention.")
    else:
        player_stats = playercareerstats.PlayerCareerStats(player_id=player_ids[player])
        player_stats_df = player_stats.get_data_frames()[0]
        player_stats_df['PLAYER_NAME'] = player
        comb_player_stats = pd.concat([comb_player_stats, player_stats_df])

Working on Lafayette Lever
Working on Bob Hansen
Working on CJ McCollum
Working on Anfernee Hardaway
Working on JR Smith
Working on Clifford Robinson
Working on World Free
Working on Steven Smith


In [27]:
comb_player_stats.to_csv('pow_data_by_yr.csv')
